#  Вложенные функции, замыкания и декораторы

## 1. Функции

**Функция** — часть программы, которую можно вызвать из другого места программы.

Все в Python объекты. И даже функции. Это значит, что у функций есть
- атрибуты
- и методы.

От остальных объектов функции отличаются тем, что их можно вызвать*. Объекты, которые можно вызвать, называют `Callable`-объектами.

*\*С точки зрения синтаксиса еще можно вызывать классы*

### Как определить функцию

In [36]:
# Функция определяется таким синтаксисом
def plus_one(x: int) -> int:
    """Функция возвращает увеличенное на 1 целое число"""
    return x+1

Это избыточное определение. Здесь использованы:
- строка документирования — docsting,
- а также анотация функции.

На самом деле можно описать эту же функцию компактней. 

In [51]:
# Функция plus_one без анотаций и документации
def plus_one_simple(x): return x+1

Как у любого объекта в python, у функции есть:
- идентификатор,
- тип.

In [47]:
# У функции plus_one эти параметры выглядят так
id(plus_one), type(plus_one)

(4510970608, function)

В CPython идентификатор — **адрес объекта** в виртуальной памяти

In [48]:
# Идентификатор в шестнадцатиричном формате — адрес функции plus_one
hex(id(plus_one))

'0x10cdff2f0'

Все атрибуты и методы функции как объекта можно посмотреть:

In [45]:
import inspect
list(filter(lambda x: x[0] != "__globals__", sorted(inspect.getmembers(plus_one))))
# Здесь мы выбросили поле "__globals__", чтобы не засорять вывод

[('__annotations__', {'x': int, 'return': int}),
 ('__call__', <method-wrapper '__call__' of function object at 0x10cdff2f0>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object plus_one at 0x10d59f780, file "<ipython-input-36-59ce0af9670a>", line 2>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x10cdff2f0>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', 'Функция возвращает увеличенное на 1 целое число'),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x10cdff2f0>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x10cdff2f0>),
 ('__get__', <method-wrapper '__get__' of function object at 0x10cdff2f0>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x10cdff2f0>),
 ('__gt__', <method-wrapper '__gt__' of function object at 0x10cdff2f0>),
 ('__hash__', <metho

### Как вызвать функцию

In [39]:
#  Вызов функции, ожидаем ответ 2
plus_one(1)

2

In [40]:
#  Можно явно вызвать метод call, ожидаем ответ 2
plus_one.__call__(1)

2

### Как функции устроены

In [41]:
# Байт-код функции function_name
plus_one.__code__.co_code

b'|\x00d\x01\x17\x00S\x00'

In [42]:
# Дизассемблированное тело функции function_name
import dis
dis.dis(plus_one)

  4           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


Если заглянуть во внутренности интерпретатора (CPython), то функция описывается следующей струтурой: https://github.com/python/cpython/blob/3.7/Include/funcobject.h

**Почитать**
1. [The Python Language Reference. Data model](https://docs.python.org/3/reference/datamodel.html#objects-values-and-types)
2. [The Python Language Reference. Inspect live objects](https://docs.python.org/3/library/inspect.html)
3. [PEP 3107 -- Function Annotations](https://www.python.org/dev/peps/pep-3107/)
4. [PEP 257 -- Docstring Conventions](https://www.python.org/dev/peps/pep-0257/)

## 2. Вложенные функции

**Вложенная функция** — функция, которая определена внутри другой функции.

### Область видимости и LEGB

https://www.google.com/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwjumtSs7prgAhWGxIsKHdS2C4QQjRx6BAgBEAU&url=https%3A%2F%2Fsebastianraschka.com%2FArticles%2F2014_python_scope_and_namespaces.html&psig=AOvVaw1Sw3kd_XY-GCz5nsD61FwU&ust=1549121745003362![image.png](attachment:image.png)

## 3. Замыкания

**Замыкание** — вложенная функция, которая использует локальные переменные внешней функции.

## 4. Декораторы

**Декоратор** — функция, которая принимает другую функцию и что-то возвращает.